# Skater Similarity & Projection Analysis

#### Projecting future player performance is extremely difficult in hockey. First, how do we predict? Player can get hurt, change teams or roles, and are simultaneously aging and developing at rates unique to each player. Second, what is performance? Whether we project points, goals, or Goals Above Replacement, ideally it is something the player has a reasonable amount of control over.

#### In the following analysis, I am going to project future performance by finding player comparables and looking at their performance in the next season. We have 10 seasons of player season to draw for, 9 of which we know how the player fared in the season after. This analysis will walk through the Python code and methodology in a step-by-step fashion. The pros of this is complete transparency, provides others an opportunity to use the code for similar analyses or improve this one. The con is it will include technical information that may turn some (lots) of people off.

#### A few notes to start off


### 1. Import packages used for the analysis

In [2]:
%matplotlib inline

import requests
import pandas as pd
import math
import pymysql as pg 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn import cross_validation
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans

/Users/colander1/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### 2. Read in data used for analyis and check it

In [3]:
conn2 = pg.connect(host='mysql.crowdscoutsports.com', user='ca_elo_games', port=3306, db='nhl_all', password='cprice31!')

crowdscout_pred = pd.read_sql("""SELECT a.*, playerBirthDate as DOB, concat("Pos-",playerPositionCode) as Position,
                                    playerHeight as Height, 
                                    case when playerShootsCatches in ('L','R') then concat("Shoots-",playerShootsCatches) else 'Shoots-L' end as Shoots_Clean 
                                    FROM `nhl_all`.`crowdscout_data_predictions` as a 
                                    INNER JOIN `nhl_all`.`hockey_roster_info` as b 
                                    ON a.shooterID = b.playerId""",con=conn2)

print crowdscout_pred.shape
crowdscout_pred.head()

(8772, 168)


,Player,shooterID,season,Cluster,ClusterName,Pos.Rank,DepthChart,Predicted.CS,Predicted.CS.RF.Scaled,Predicted.CS.LM.Scaled,...,Player.Position,Shoots,shooterDOB,TOI,G60,P60,DOB,Position,Height,Shoots_Clean
0,AARON DOWNEY,8465992,20072008,7,Depth Defensive Forward,534.0,Other Fwd,13.737325,15.091377,12.383272,...,R,R,1974-08-27,4.431806,0.000000,0.451283,1974-08-27,Pos-R,73,Shoots-R
1,AARON JOHNSON,8469534,20072008,10,Matchup Dependent Defensive Depth,166.0,3P D,39.179558,38.230001,40.129114,...,D,L,1983-04-30,6.990278,0.000000,0.286112,1983-04-30,Pos-D,73,Shoots-L
2,AARON MILLER,8457384,20072008,10,Matchup Dependent Defensive Depth,220.0,Other D,32.443200,27.655049,37.231352,...,D,R,1971-08-11,16.349861,0.061163,0.428138,1971-08-11,Pos-D,75,Shoots-R
3,AARON ROME,8470310,20072008,10,Matchup Dependent Defensive Depth,212.0,Other D,33.067929,25.822331,40.313527,...,D,L,1983-09-27,5.019028,0.199242,0.398484,1983-09-27,Pos-D,73,Shoots-L
4,AARON VOROS,8469672,20072008,4,Matchup Capable Offensive Depth,293.0,4L Fwd,35.554521,33.154615,37.954427,...,L,L,1981-07-02,9.588889,0.834299,1.564311,1981-07-02,Pos-L,74,Shoots-L


In [4]:
current_roster = pd.read_sql("""SELECT a.nhl_id as PlayerID, a.player_name as player_name, a.pos as Pos,
                                case when a.pos = 'D' then 'D' else 'F' end as player_position,
                                a.team as player_team1, round(DATEDIFF(current_date(),a.dob)/365.25,1) as age1, 
                                a.height, a.weight 
                                FROM hockey_roster_v1 as a
                                WHERE pos != 'G'""",con=conn2)

current_roster.head()

,PlayerID,player_name,Pos,player_position,player_team1,age1,height,weight
0,8448208,Jaromir Jagr,RW,F,FLA,45.4,"6' 3""",230
1,8462038,Shane Doan,RW,F,ARI,40.8,"6' 1""",223
2,8462042,Jarome Iginla,RW,F,LAK,40.0,"6' 1""",210
3,8464989,Matt Cullen,C,F,PIT,40.7,"6' 1""",200
4,8465009,Zdeno Chara,D,D,BOS,40.3,"6' 9""",250


### 3. Create an age variable, player age to start the season

In [5]:
### Create Age Variable
crowdscout_pred['Season_Age'] = (pd.to_datetime(crowdscout_pred.season.str[0:4] + "-09-20") - pd.to_datetime(crowdscout_pred.DOB)).astype('timedelta64[D]') 

crowdscout_pred['Season_Age'] = (crowdscout_pred['Season_Age'] / 365.25).round(1)

crowdscout_pred['Season_Age'].head()

0    33.1
1    24.4
2    36.1
3    24.0
4    26.2
Name: Season_Age, dtype: float64

### 4. Create dummy variables for player position, handedness, and player cluster

In [6]:
### Create Position Dummy Variables
crowdscout_pred_df = pd.concat([crowdscout_pred,pd.get_dummies(crowdscout_pred['Position'])\
                                ,pd.get_dummies(crowdscout_pred['Shoots_Clean'])\
                                ,pd.get_dummies(crowdscout_pred['ClusterName'])], axis=1)

crowdscout_pred_df = crowdscout_pred_df.set_index(['season','shooterID'])

crowdscout_pred_df.head()

Player  Cluster                        ClusterName  \
season   shooterID                                                              
20072008 8465992     AARON DOWNEY        7            Depth Defensive Forward   
         8469534    AARON JOHNSON       10  Matchup Dependent Defensive Depth   
         8457384     AARON MILLER       10  Matchup Dependent Defensive Depth   
         8470310       AARON ROME       10  Matchup Dependent Defensive Depth   
         8469672      AARON VOROS        4    Matchup Capable Offensive Depth   

                    Pos.Rank DepthChart  Predicted.CS  Predicted.CS.RF.Scaled  \
season   shooterID                                                              
20072008 8465992       534.0  Other Fwd     13.737325               15.091377   
         8469534       166.0       3P D     39.179558               38.230001   
         8457384       220.0    Other D     32.443200               27.655049   
         8470310       212.0    Other D     33.067929               25.822331   
         8469672       293.0     4L Fwd     35.554521               33.154615   

                    Predicted.CS.LM.Scaled Pos  Games.Played_EV  \
season   shooterID                                                
20072008 8465992                 12.383272   F             56.0   
         8469534                 40.129114   D             30.0   
         8457384                 37.231352   D             57.0   
         8470310                 40.313527   D             17.0   
         8469672                 37.954427   F             60.0   

                                       ...                      \
season   shooterID                     ...                       
20072008 8465992                       ...                       
         8469534                       ...                       
         8457384                       ...                       
         8470310                       ...                       
         8469672                       ...                       

                    All-Around Skilled Defensive Driver  \
season   shooterID                                        
20072008 8465992                                      0   
         8469534                                      0   
         8457384                                      0   
         8470310                                      0   
         8469672                                      0   

                    All-Around Skilled Offensive Driver  Defensive Depth  \
season   shooterID                                                         
20072008 8465992                                      0                0   
         8469534                                      0                0   
         8457384                                      0                0   
         8470310                                      0                0   
         8469672                                      0                0   

                    Depth Defensive Forward  \
season   shooterID                            
20072008 8465992                          1   
         8469534                          0   
         8457384                          0   
         8470310                          0   
         8469672                          0   

                    Matchup Capable Defensive Forward  \
season   shooterID                                      
20072008 8465992                                    0   
         8469534                                    0   
         8457384                                    0   
         8470310                                    0   
         8469672                                    0   

                    Matchup Capable Defensive Player  \
season   shooterID                                     
20072008 8465992                                   0   
         8469534                                   0   
         8457384                                   0   
 

In [7]:
## Set variables to be scaled
scale_vars = ["Season_Age",
              "Pos-C","Pos-D","Pos-L","Pos-R",
              
              "Total.Shifts_EV","Total.Shifts_PP","Total.Shifts_SH","OTF.Shift.Share_EV","OTF.Shift.Share_PP",
              "OTF.Shift.Share_SH",
                "Off.FO.Shift.Share_EV","Off.FO.Shift.Share_PP","Off.FO.Shift.Share_SH","Def.FO.Shift.Share_EV",
              "Def.FO.Shift.Share_PP","Def.FO.Shift.Share_SH",
                "ixG60_EV","ixG60_PP","ixG60_SH","G60_EV","G60_PP","G60_SH",             
                "A160_EV","A160_PP","A160_SH","xGF60_EV","xGF60_PP","xGF60_SH",           
                "xGA60_EV","xGA60_PP","xGA60_SH","Player_Competition_EV","Player_Teammates_EV","Player_Teammates_PP",
                "Share.of.Ice_EV","Share.of.Ice_PP","Share.of.Ice_SH","xGF60_Rel_EV","xGF60_Rel_PP","xGF60_Rel_SH",
                "xGA60_Rel_EV","xGA60_Rel_PP","xGA60_Rel_SH","P60_EV","P60_PP","P60_SH",
                "Teammates_Diff_EV","Teammates_Diff_PP","Pos.Rank",

              "Depth Defensive Forward","Matchup Dependent Defensive Depth","Matchup Capable Offensive Depth",
                "Matchup Capable Defensive Player","Defensive Depth","All-Around Skilled Defensive Driver",
                "All-Around Skilled Offensive Driver","All-Around Matchup Capable Offensive Driver","Matchup Dependent Skilled Defensive Player",
                "Matchup Capable Skilled Offensive Driver","Matchup Capable Defensive Forward"]


crowdscout_pred_df = crowdscout_pred_df[scale_vars]

crowdscout_pred_df.head()

Season_Age  Pos-C  Pos-D  Pos-L  Pos-R  Total.Shifts_EV  \
season   shooterID                                                            
20072008 8465992          33.1      0      0      0      1            419.0   
         8469534          24.4      0      1      0      0            544.0   
         8457384          36.1      0      1      0      0           1248.0   
         8470310          24.0      0      1      0      0            389.0   
         8469672          26.2      0      0      1      0            857.0   

                    Total.Shifts_PP  Total.Shifts_SH  OTF.Shift.Share_EV  \
season   shooterID                                                         
20072008 8465992                3.0              3.0            0.548926   
         8469534               85.0             29.0            0.485294   
         8457384               25.0            220.0            0.494391   
         8470310               30.0             39.0            0.478149   
         8469672              137.0              7.0            0.474912   

                    OTF.Shift.Share_PP                ...                  \
season   shooterID                                    ...                   
20072008 8465992              0.000000                ...                   
         8469534              0.341176                ...                   
         8457384              0.480000                ...                   
         8470310              0.433333                ...                   
         8469672              0.299270                ...                   

                    Matchup Dependent Defensive Depth  \
season   shooterID                                      
20072008 8465992                                    0   
         8469534                                    1   
         8457384                                    1   
         8470310                                    1   
         8469672                                    0   

                    Matchup Capable Offensive Depth  \
season   shooterID                                    
20072008 8465992                                  0   
         8469534                                  0   
         8457384                                  0   
         8470310                                  0   
         8469672                                  1   

                    Matchup Capable Defensive Player  Defensive Depth  \
season   shooterID                                                      
20072008 8465992                                   0                0   
         8469534                                   0                0   
         8457384                                   0                0   
         8470310                                   0                0   
         8469672                                   0                0   

                    All-Around Skilled Defensive Driver  \
season   shooterID                                        
20072008 8465992                                      0   
         8469534                                      0   
         8457384                                      0   
         8470310                                      0   
         8469672                                      0   

                    All-Around Skilled Offensive Driver  \
season   shooterID                                        
20072008 8465992                                      0   
         8469534                                      0   
         8457384                                      0   
         8470310                                      0   
         8469672                                      0   

                    All-Around Matchup Capable Offensive Driver  \
season   shooterID                                                
20072008 8465992                                              0   
         8469534                                              0   

In [8]:
## Create function to scale and center data by season
def season_scaled_fun(year):
    ## Subset results and scale
    season_df = crowdscout_pred_df.loc[ year ,scale_vars]

    season_scaled_df = pd.DataFrame(preprocessing.scale(season_df),
                                    columns = scale_vars)

    ## Verify variance is uniform
    print season_scaled_df.mean(axis=0).mean(axis=0)
    print season_scaled_df.std(axis=0).mean(axis=0)

    ## Reset indicies
    season_scaled_df['season'] = year
    season_scaled_df['shooterID'] = season_df.index
    season_scaled_df = season_scaled_df.set_index(['shooterID','season'])

    return season_scaled_df


In [9]:
crowdscout_pred_scaled = pd.DataFrame()

for i in set(crowdscout_pred['season']):
    print i
    data = season_scaled_fun(i)
    print data.shape
    crowdscout_pred_scaled = crowdscout_pred_scaled.append(data)
    
print crowdscout_pred_scaled.shape
print crowdscout_pred_scaled.mean(axis=0).mean(axis=0)
print crowdscout_pred_scaled.std(axis=0).mean(axis=0)


20112012
8.07260882197e-18
1.00055850325
(896, 61)
20082009
7.79499727485e-17
1.0005712654
(876, 61)
20092010
9.45669069746e-17
1.0005712654
(876, 61)
20072008
9.17407020656e-17
1.00059577009
(840, 61)
20162017
1.09424879628e-17
1.00056609119
(884, 61)
20122013
-6.82578924509e-17
1.00059364802
(843, 61)
20102011
1.20890479377e-17
1.00056227162
(890, 61)
20142015
-2.64530464721e-17
1.00056866652
(880, 61)
20152016
8.48060174504e-18
1.00055601895
(900, 61)
20132014
-3.60373116299e-17
1.00056417494
(887, 61)
(8772, 61)
1.72171431425e-17
1.00005700442


In [10]:
print crowdscout_pred_scaled.shape
crowdscout_pred_scaled.head()

(8772, 61)


,,Season_Age,Pos-C,Pos-D,Pos-L,Pos-R,Total.Shifts_EV,Total.Shifts_PP,Total.Shifts_SH,OTF.Shift.Share_EV,OTF.Shift.Share_PP,...,Matchup Dependent Defensive Depth,Matchup Capable Offensive Depth,Matchup Capable Defensive Player,Defensive Depth,All-Around Skilled Defensive Driver,All-Around Skilled Offensive Driver,All-Around Matchup Capable Offensive Driver,Matchup Dependent Skilled Defensive Player,Matchup Capable Skilled Offensive Driver,Matchup Capable Defensive Forward
shooterID,season,,,,,,,,,,,,,,,,,,,,,
8471451,20112012,-0.288042,1.657997,-0.702377,-0.515303,-0.489163,-1.301424,-0.840548,-0.826434,1.031451,-1.402422,...,3.238391,-0.383728,-0.315256,-0.319517,-0.295599,-0.293361,-0.321634,-0.284289,-0.297825,-0.295599
8469534,20112012,0.372981,-0.603137,1.423737,-0.515303,-0.489163,0.145725,-0.325964,0.002986,0.182932,0.633651,...,-0.308795,-0.383728,3.172023,-0.319517,-0.295599,-0.293361,-0.321634,-0.284289,-0.297825,-0.295599
8474604,20112012,-1.191440,-0.603137,1.423737,-0.515303,-0.489163,-1.159102,-0.813104,-0.781601,0.221582,-0.245562,...,3.238391,-0.383728,-0.315256,-0.319517,-0.295599,-0.293361,-0.321634,-0.284289,-0.297825,-0.295599
8474030,20112012,-1.037202,-0.603137,-0.702377,-0.515303,2.044307,-0.855566,-0.792520,-0.826434,1.015971,1.902891,...,-0.308795,-0.383728,-0.315256,-0.319517,-0.295599,-0.293361,-0.321634,-0.284289,-0.297825,-0.295599
8470310,20112012,0.284845,-0.603137,1.423737,-0.515303,-0.489163,-0.243456,-0.518076,-0.378099,0.725149,0.763613,...,-0.308795,-0.383728,-0.315256,-0.319517,-0.295599,-0.293361,-0.321634,3.517547,-0.297825,-0.295599


In [11]:
from scipy.spatial.distance import pdist, squareform

distances = pdist(crowdscout_pred_scaled.values, metric='euclidean')
DistMatrix = pd.DataFrame(squareform(distances))

In [12]:

DistMatrix.index = crowdscout_pred_scaled.index

DistMatrix = DistMatrix.T

DistMatrix.index = crowdscout_pred_scaled.index

DistMatrix.head()

,shooterID,8471451,8469534,8474604,8474030,8470310,8475619,8470063,8471490,8467925,8474641,...,8470610,8474743,8475119,8475902,8475178,8474250,8469760,8465009,8476878,8469820
,season,20112012,20112012,20112012,20112012,20112012,20112012,20112012,20112012,20112012,20112012,...,20132014,20132014,20132014,20132014,20132014,20132014,20132014,20132014,20132014,20132014
shooterID,season,,,,,,,,,,,,,,,,,,,,,
8471451,20112012,0.000000,8.464686,6.767277,9.182239,9.297158,10.002610,8.970840,13.803371,9.934020,10.876870,...,13.656532,5.921901,9.923544,7.484916,9.554570,8.518320,9.933683,12.376903,9.061936,8.959647
8469534,20112012,8.464686,0.000000,6.853800,8.782740,6.172699,9.680162,7.044930,13.931689,8.390149,8.686982,...,11.083589,6.595908,9.903249,8.327062,7.362439,7.413187,4.747507,8.599256,7.291291,9.269410
8474604,20112012,6.767277,6.853800,0.000000,8.454673,7.034662,8.940039,8.124069,14.103450,9.437820,9.958340,...,12.089764,4.899121,9.287277,5.452101,8.509936,8.421346,8.013660,10.675477,8.121550,9.679922
8474030,20112012,9.182239,8.782740,8.454673,0.000000,8.555583,8.202073,8.589261,10.903571,9.968462,11.428165,...,14.398318,8.374480,9.679481,9.233496,7.699835,9.768444,10.888343,13.702043,9.117770,10.368222
8470310,20112012,9.297158,6.172699,7.034662,8.555583,0.000000,9.293515,7.850199,13.770069,9.517815,9.659019,...,11.927897,7.404430,9.754811,8.181796,7.909802,8.301660,8.410510,10.025506,8.102725,9.658196


In [22]:
## Find all seasons 
seasons = crowdscout_pred.season.sort_values(ascending=True).drop_duplicates()

## List of compare years and most current season
comp_years = list(seasons.reset_index(drop=True).iloc[:-1])
current_year = list(seasons.reset_index(drop=True).iloc[2:])

print comp_years
print current_year

CompsMat = DistMatrix.loc[DistMatrix.index.isin(comp_years, level="season"),DistMatrix.index.isin(current_year, level="season")]

print CompsMat.shape
CompsMat.head()

['20072008', '20082009', '20092010', '20102011', '20112012', '20122013', '20132014', '20142015', '20152016']
['20092010', '20102011', '20112012', '20122013', '20132014', '20142015', '20152016', '20162017']
(7888, 7056)


,shooterID,8471451,8469534,8474604,8474030,8470310,8475619,8470063,8471490,8467925,8474641,...,8470610,8474743,8475119,8475902,8475178,8474250,8469760,8465009,8476878,8469820
,season,20112012,20112012,20112012,20112012,20112012,20112012,20112012,20112012,20112012,20112012,...,20132014,20132014,20132014,20132014,20132014,20132014,20132014,20132014,20132014,20132014
shooterID,season,,,,,,,,,,,,,,,,,,,,,
8471451,20112012,0.000000,8.464686,6.767277,9.182239,9.297158,10.002610,8.970840,13.803371,9.934020,10.876870,...,13.656532,5.921901,9.923544,7.484916,9.554570,8.518320,9.933683,12.376903,9.061936,8.959647
8469534,20112012,8.464686,0.000000,6.853800,8.782740,6.172699,9.680162,7.044930,13.931689,8.390149,8.686982,...,11.083589,6.595908,9.903249,8.327062,7.362439,7.413187,4.747507,8.599256,7.291291,9.269410
8474604,20112012,6.767277,6.853800,0.000000,8.454673,7.034662,8.940039,8.124069,14.103450,9.437820,9.958340,...,12.089764,4.899121,9.287277,5.452101,8.509936,8.421346,8.013660,10.675477,8.121550,9.679922
8474030,20112012,9.182239,8.782740,8.454673,0.000000,8.555583,8.202073,8.589261,10.903571,9.968462,11.428165,...,14.398318,8.374480,9.679481,9.233496,7.699835,9.768444,10.888343,13.702043,9.117770,10.368222
8470310,20112012,9.297158,6.172699,7.034662,8.555583,0.000000,9.293515,7.850199,13.770069,9.517815,9.659019,...,11.927897,7.404430,9.754811,8.181796,7.909802,8.301660,8.410510,10.025506,8.102725,9.658196


In [23]:
print type(CompsMat)
#player_data = CompsMat.loc[:,CompsMat.index.isin(['8471675'], level="shooterID")]
player_data = CompsMat.T.loc[:,['8471675']]

print player_data.shape
player_data.head()

<class 'pandas.core.frame.DataFrame'>
(7056, 9)


shooterID             8471675                                              \
season               20112012   20082009   20092010   20072008   20122013   
shooterID season                                                            
8471451   20112012  14.417659  13.504916  13.491601  13.970272  13.493075   
8469534   20112012  12.290372  11.103948  11.487699  12.157366  11.453002   
8474604   20112012  13.556967  12.532253  12.835380  13.137987  13.203633   
8474030   20112012  15.279862  14.187545  14.555904  14.988229  14.927995   
8470310   20112012  13.014144  11.971843  12.486071  12.835072  12.742420   

shooterID                                                       
season               20102011   20142015   20152016   20132014  
shooterID season                                                
8471451   20112012  13.851293  12.190457  12.991244  12.663840  
8469534   20112012  11.794350  10.043566  10.575194  10.645945  
8474604   20112012  13.541574  11.552356  12.291668  12.280710  
8474030   20112012  14.969927  13.167338  14.097664  14.262591  
8470310   20112012  12.743611  10.959349  11.640643  11.985225

In [24]:
def player_comps(ID):
    ## Subset Player Columns
    try:
        player_data = CompsMat.loc[:,str(ID)]
        
        print player_data.shape

        ## Empty DF
        player_comps = pd.DataFrame()

        ## Loop through each player season
        for szn in set(player_data.T.index):

            ## Find 15 closest comparables
            player_season = player_data[szn].sort_values(ascending = True).head(40).rename("SimilarityScore")

            ## Remove same season
            player_season = pd.DataFrame(player_season[0:])

            player_season['PlayerID'] = ID
            player_season['PlayerSeason'] = szn

            player_season.reset_index(inplace=True)  

            player_season = player_season.rename(index=str, columns={"shooterID": "CompID", "season": "CompSeason"})   

            ## Append to Empty DF
            player_comps = player_comps.append(player_season)

        return player_comps
    except Exception:
        pass

In [25]:
player_comps('8471675')

(7888, 8)


,CompID,CompSeason,SimilarityScore,PlayerID,PlayerSeason
0,8471675,20152016,0.000000,8471675,20152016
1,8471675,20132014,2.868532,8471675,20152016
2,8470794,20152016,2.871713,8471675,20152016
3,8471675,20142015,3.345140,8471675,20152016
4,8477497,20152016,3.685801,8471675,20152016
5,8471675,20082009,3.755478,8471675,20152016
6,8475794,20152016,3.807806,8471675,20152016
7,8471215,20152016,3.816012,8471675,20152016
8,8471215,20072008,3.875293,8471675,20152016
9,8469455,20152016,3.926005,8471675,20152016


In [29]:
### Loop through each playerID finding comps
player_comp_df = pd.DataFrame()

for i in set(current_roster['PlayerID']):
    
    data = player_comps(i)
    player_comp_df = player_comp_df.append(data)
        

(7888, 6)
(7888, 8)
(7888, 6)
(7888, 8)
(7888, 8)
(7888, 8)
(7888, 7)
(7888, 8)
(7888, 3)
(7888, 3)
(7888, 8)
(7888, 8)
(7888, 8)
(7888, 8)
(7888, 4)
(7888, 7)
(7888, 5)
(7888, 7)
(7888, 8)
(7888, 7)
(7888, 8)
(7888, 4)
(7888, 8)
(7888, 8)
(7888, 8)
(7888, 8)
(7888, 8)
(7888, 8)
(7888, 8)
(7888, 8)
(7888, 8)
(7888, 6)
(7888, 8)
(7888, 8)
(7888, 8)
(7888, 8)
(7888, 6)
(7888, 7)
(7888, 8)
(7888, 8)
(7888, 7)
(7888, 3)
(7888, 8)
(7888, 7)
(7888, 1)
(7888, 4)
(7888, 6)
(7888, 8)
(7888, 1)
(7888, 8)
(7888, 7)
(7888, 6)
(7888, 1)
(7888, 7)
(7888, 8)
(7888, 8)
(7888, 5)
(7888, 4)
(7888, 5)
(7888, 8)
(7888, 2)
(7888, 6)
(7888, 8)
(7888, 4)
(7888, 5)
(7888, 3)
(7888, 4)
(7888, 4)
(7888, 5)
(7888, 4)
(7888, 2)
(7888, 1)
(7888, 8)
(7888, 2)
(7888, 8)
(7888, 2)
(7888, 5)
(7888, 4)
(7888, 2)
(7888, 4)
(7888, 4)
(7888, 6)
(7888, 4)
(7888, 4)
(7888, 8)
(7888, 4)
(7888, 5)
(7888, 5)
(7888, 2)
(7888, 1)
(7888, 2)
(7888, 8)
(7888, 5)
(7888, 8)
(7888, 2)
(7888, 8)
(7888, 3)
(7888, 8)
(7888, 2)
(7888, 5)


In [30]:
##### Construct 

player_comp_df['PlayerID'] = player_comp_df['PlayerID'].apply(str)

print player_comp_df.shape
print player_comp_df.dtypes
player_comp_df.head()

(146960, 5)
CompID              object
CompSeason          object
SimilarityScore    float64
PlayerID            object
PlayerSeason        object
dtype: object


,CompID,CompSeason,SimilarityScore,PlayerID,PlayerSeason
0,8474625,20112012,0.000000,8474625,20112012
1,8475761,20122013,3.737916,8474625,20112012
2,8465026,20082009,3.836689,8474625,20112012
3,8471263,20072008,4.355367,8474625,20112012
4,8474114,20082009,4.379913,8474625,20112012


In [31]:
## Collapse player season scores
player_season_scores = crowdscout_pred.groupby(['shooterID','season','Player'])[['Predicted.CS']].mean()

player_season_scores.reset_index(inplace=True)  

## should be the same
print crowdscout_pred.shape
print player_season_scores.shape
player_season_scores.head()

(8772, 169)
(8772, 4)


,shooterID,season,Player,Predicted.CS
0,8445550,20072008,ROB BLAKE,51.804668
1,8445550,20082009,ROB BLAKE,74.730882
2,8445550,20092010,ROB BLAKE,67.332515
3,8445735,20072008,ROD BRIND'AMOUR,75.352488
4,8445735,20082009,ROD BRIND'AMOUR,63.179548


In [32]:
#player_season_scores.loc[player_season_scores['shooterID'].isin(['8471675']),:]

player_season_scores.loc[player_season_scores['shooterID'].isin(['8474625']),:]

,shooterID,season,Player,Predicted.CS
6513,8474625,20112012,JIMMY HAYES,32.919228
6514,8474625,20122013,JIMMY HAYES,36.186527
6515,8474625,20132014,JIMMY HAYES,40.849004
6516,8474625,20142015,JIMMY HAYES,59.738470
6517,8474625,20152016,JIMMY HAYES,45.063627
6518,8474625,20162017,JIMMY HAYES,29.917903


### Match Comparable Scores

In [33]:
comp_match_scores = crowdscout_pred.loc[:,['shooterID','season','Player','Predicted.CS']]
comp_match_scores.columns = ['CompID','CompSeason','Comparable','CompScore']

print comp_match_scores.dtypes
print player_comp_df.dtypes

player_comp_df_v1 = player_comp_df.merge(comp_match_scores, how='left', on=['CompID','CompSeason'])

## should be same rows
print player_comp_df.shape
print player_comp_df_v1.shape
player_comp_df_v1.head()

CompID         object
CompSeason     object
Comparable     object
CompScore     float64
dtype: object
CompID              object
CompSeason          object
SimilarityScore    float64
PlayerID            object
PlayerSeason        object
dtype: object
(146960, 5)
(146960, 7)


,CompID,CompSeason,SimilarityScore,PlayerID,PlayerSeason,Comparable,CompScore
0,8474625,20112012,0.000000,8474625,20112012,JIMMY HAYES,32.919228
1,8475761,20122013,3.737916,8474625,20112012,BEAU BENNETT,44.044700
2,8465026,20082009,3.836689,8474625,20112012,MARK PARRISH,43.932872
3,8471263,20072008,4.355367,8474625,20112012,ENVER LISIN,42.523236
4,8474114,20082009,4.379913,8474625,20112012,OSCAR MOLLER,48.181589


## Match Comparable Plus 1 Year Scores

In [34]:
comp_match_next_scores = crowdscout_pred.loc[:,['shooterID','season','Predicted.CS','TOI','P60','G60','P60_EV','G60_EV','P60_PP','G60_PP']]

comp_match_next_scores.columns = ['CompID','CompSeason','CompP1_Score','CompP1_TOI','CompP1_P60','CompP1_G60','CompP1_P60_EV','CompP1_G60_EV','CompP1_P60_PP','CompP1_G60_PP']

comp_match_next_scores['CompSeason'] = (comp_match_next_scores['CompSeason'].astype(int) - 10001).astype(str)


player_comp_df_v2 = player_comp_df_v1.merge(comp_match_next_scores, how='left', on=['CompID','CompSeason'])

## should be same rows
print player_comp_df_v1.shape
print player_comp_df_v2.shape
player_comp_df_v2.head()

(146960, 7)
(146960, 15)


,CompID,CompSeason,SimilarityScore,PlayerID,PlayerSeason,Comparable,CompScore,CompP1_Score,CompP1_TOI,CompP1_P60,CompP1_G60,CompP1_P60_EV,CompP1_G60_EV,CompP1_P60_PP,CompP1_G60_PP
0,8474625,20112012,0.000000,8474625,20112012,JIMMY HAYES,32.919228,36.186527,2.351250,1.701223,0.425306,1.788153,0.447038,0.000000,0.000000
1,8475761,20122013,3.737916,8474625,20112012,BEAU BENNETT,44.044700,42.709845,7.324861,1.638256,0.546085,1.471640,0.441492,2.060086,2.060086
2,8465026,20082009,3.836689,8474625,20112012,MARK PARRISH,43.932872,34.082270,3.901111,0.256337,0.000000,0.319333,0.000000,0.000000,0.000000
3,8471263,20072008,4.355367,8474625,20112012,ENVER LISIN,42.523236,55.212640,11.724722,1.791087,1.108768,1.900683,1.161529,1.124297,0.562149
4,8474114,20082009,4.379913,8474625,20112012,OSCAR MOLLER,48.181589,30.353453,4.890000,1.431493,0.817996,1.216052,0.729631,2.775636,1.387818


## Match Player Scores

In [35]:
player_match_scores = crowdscout_pred.loc[:,['shooterID','season','Player','Predicted.CS']]
player_match_scores.columns = ['PlayerID','PlayerSeason','Player','PlayerScore']

#print player_match_scores.dtypes
#print player_comp_df_v1.dtypes

player_comp_df_v3 = player_comp_df_v2.merge(player_match_scores, how='left', on=['PlayerID','PlayerSeason'])

player_comp_df_v3.head()

,CompID,CompSeason,SimilarityScore,PlayerID,PlayerSeason,Comparable,CompScore,CompP1_Score,CompP1_TOI,CompP1_P60,CompP1_G60,CompP1_P60_EV,CompP1_G60_EV,CompP1_P60_PP,CompP1_G60_PP,Player,PlayerScore
0,8474625,20112012,0.000000,8474625,20112012,JIMMY HAYES,32.919228,36.186527,2.351250,1.701223,0.425306,1.788153,0.447038,0.000000,0.000000,JIMMY HAYES,32.919228
1,8475761,20122013,3.737916,8474625,20112012,BEAU BENNETT,44.044700,42.709845,7.324861,1.638256,0.546085,1.471640,0.441492,2.060086,2.060086,JIMMY HAYES,32.919228
2,8465026,20082009,3.836689,8474625,20112012,MARK PARRISH,43.932872,34.082270,3.901111,0.256337,0.000000,0.319333,0.000000,0.000000,0.000000,JIMMY HAYES,32.919228
3,8471263,20072008,4.355367,8474625,20112012,ENVER LISIN,42.523236,55.212640,11.724722,1.791087,1.108768,1.900683,1.161529,1.124297,0.562149,JIMMY HAYES,32.919228
4,8474114,20082009,4.379913,8474625,20112012,OSCAR MOLLER,48.181589,30.353453,4.890000,1.431493,0.817996,1.216052,0.729631,2.775636,1.387818,JIMMY HAYES,32.919228


## Match Next Season Player Scores/Results

In [36]:
player_match_next_scores = crowdscout_pred.loc[:,['shooterID','season','Predicted.CS','TOI','P60','G60']]

player_match_next_scores.columns = ['PlayerID','Season','PlayerP1_Score','PlayerP1_TOI','PlayerP1_P60','PlayerP1_G60']

## Set Season Back One Season
player_match_next_scores['Season'] = (player_match_next_scores['Season'].astype(int) - 10001).astype(str)


#player_comp_df_v2 = player_comp_df_v1.merge(comp_match_next_scores, how='left', on=['CompID','CompSeason'])

## should be same rows
#print player_comp_df_v1.shape
#print player_comp_df_v2.shape
#player_comp_df_v2.head()

## Create function to check tuning parameters

#### Find parameters to use to forecast to metrics, Points/60 and Predicted Score. Parameters include number of comparables, weighting of comparables, dissimilarity range, marcels, marcel years, and metric

In [37]:
def weighting_tuning(metric, num_comps, dissim_range, marcel_years, marcel_decay):
    import scipy
    
    player_matches = pd.DataFrame()

    data_df = player_comp_df_v3
    
    try:
        for season in ['20102011', '20112012', '20122013', '20132014', '20142015', '20152016']:

            data = data_df
            
            data['comp_seasons_prior'] = data['PlayerSeason'].str[-4:].astype(int) - data['CompSeason'].str[-4:].astype(int)

            player_top_matches = data.loc[data['comp_seasons_prior'] >= 0, :].\
                        sort_values(['Player','PlayerID','PlayerSeason','PlayerScore','SimilarityScore'],ascending = True).\
                        groupby(['Player','PlayerID','PlayerSeason','PlayerScore']).head(num_comps)

            ## Keep last X years
            player_top_matches['player_seasons_prior'] = (int(season[-4:]) - player_top_matches['PlayerSeason'].str[-4:].astype(int))

            ## Keep Prior Season in Range
            player_top_matches_v2 = player_top_matches.loc[player_top_matches['player_seasons_prior'] < marcel_years,:].\
                                                   loc[player_top_matches['player_seasons_prior'] >= 0,:]

            ## Create season weight
            player_top_matches_v2['season_weight'] = marcel_decay ** player_top_matches_v2['player_seasons_prior']

            ## Calculate weight
            player_top_matches_v2['comp_weight'] = ((dissim_range - player_top_matches_v2['SimilarityScore']) / dissim_range)\
                                          * player_top_matches_v2['season_weight']

            ## Weight metric
            player_top_matches_v2['weighted_metric'] = player_top_matches_v2['CompP1_' + metric]\
                                          * player_top_matches_v2['comp_weight']

            player_top_matches_v2['Season'] = season

            player_level = player_top_matches_v2.groupby(['Player','PlayerID','Season'], as_index=False)\
                                            ['weighted_metric','comp_weight'].sum()

            player_level['Projected_' + metric] = player_level['weighted_metric'] / player_level['comp_weight']

            player_level_v2 = player_level.merge(player_match_next_scores, how='inner', on=['PlayerID','Season'])

            player_matches = player_matches.append(player_level_v2) ## store dataframes in list
                    
        player_matches = player_matches.loc[:,['Projected_' + metric, 'PlayerP1_' + metric]].dropna()

        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(player_matches['Projected_' + metric],player_matches['PlayerP1_' + metric])
        return r_value**2
    
    except Exception:
        pass    

In [38]:
#print weighting_tuning('Score',15,30,2,0.5)

#print weighting_tuning('Score',15,30,2,0.7)

print weighting_tuning('P60',20.0,40.0,4.0,0.33)
#correlations

0.586533194406


In [39]:
expand_grid = pd.DataFrame()

from itertools import product

def expand_grid(dictionary):
   return pd.DataFrame([row for row in product(*dictionary.values())], 
                       columns=dictionary.keys())

dictionary = {'metric': ['P60','Score','TOI','G60'],
               'num_comps': [3, 5, 10, 15, 20],
               'dissim_range': [20, 30, 40, 50],
               'marcel_years': [1, 2, 3, 4, 5],    
               'marcel_decay': [0.5, 0.75, 0.33]}

grid = expand_grid(dictionary)


grid = grid[['metric','num_comps','dissim_range','marcel_years','marcel_decay']]

print grid.shape

print grid.head()

(1200, 5)
  metric  num_comps  dissim_range  marcel_years  marcel_decay
0    P60          3            20             1          0.50
1    P60          3            30             1          0.50
2    P60          3            40             1          0.50
3    P60          3            50             1          0.50
4    P60          3            20             1          0.75


In [40]:
rsquareds = pd.DataFrame()

for i in range(grid['metric'].count()):
    
    rsquared = weighting_tuning(metric = grid.iloc[i,0],
                           num_comps = grid.iloc[i,1], 
                           dissim_range = grid.iloc[i,2], 
                           marcel_years = grid.iloc[i,3], 
                           marcel_decay = grid.iloc[i,4])
    
    print str(rsquared) + " " + str(grid.iloc[i,0]) + " " + str(grid.iloc[i,1]) +\
        " " + str(grid.iloc[i,2]) + " " + str(grid.iloc[i,3]) + " " + str(grid.iloc[i,4])
        
    rsquared = pd.Series([rsquared, grid.iloc[i,0], grid.iloc[i,1], grid.iloc[i,2], grid.iloc[i,3], grid.iloc[i,4]])
    
    rsquareds = rsquareds.append(rsquared, ignore_index=True)

print rsquareds.shape

0.762267174449 P60 3 20 1 0.5
0.748586041793 P60 3 30 1 0.5
0.742202629526 P60 3 40 1 0.5
0.738519033723 P60 3 50 1 0.5
0.762267174449 P60 3 20 1 0.75
0.748586041793 P60 3 30 1 0.75
0.742202629526 P60 3 40 1 0.75
0.738519033723 P60 3 50 1 0.75
0.762267174449 P60 3 20 1 0.33
0.748586041793 P60 3 30 1 0.33
0.742202629526 P60 3 40 1 0.33
0.738519033723 P60 3 50 1 0.33
0.684041647716 P60 3 20 2 0.5
0.674672684039 P60 3 30 2 0.5
0.670273970807 P60 3 40 2 0.5
0.667726954972 P60 3 50 2 0.5
0.673320317063 P60 3 20 2 0.75
0.664505868257 P60 3 30 2 0.75
0.66036693093 P60 3 40 2 0.75
0.657969617456 P60 3 50 2 0.75
0.690823860657 P60 3 20 2 0.33
0.680901143218 P60 3 30 2 0.33
0.676244462425 P60 3 40 2 0.33
0.673549289598 P60 3 50 2 0.33
0.679110586907 P60 3 20 3 0.5
0.670197330624 P60 3 30 3 0.5
0.666018705351 P60 3 40 3 0.5
0.663599569948 P60 3 50 3 0.5
0.661825499699 P60 3 20 3 0.75
0.653579756526 P60 3 30 3 0.75
0.649712244726 P60 3 40 3 0.75
0.647472561551 P60 3 50 3 0.75
0.688920615173 P60 3 

In [41]:
rsquareds.columns = ['rsquared','metric','num_comps','dissim_range','marcel_years','marcel_decay']

top_metrics_v1 = top_metrics
print top_metrics_v1
top_metrics = rsquareds.sort_values(['metric','rsquared'],ascending = False).groupby(['metric']).head(2)

top_metrics

NameError: name 'top_metrics' is not defined

### Baseline Year 1 - Year 2 Residuals

In [ ]:
player_season_stats = crowdscout_pred.loc[:,['Player','shooterID','season','Predicted.CS','TOI','P60','G60']]
player_season_stats_p1 = crowdscout_pred.loc[:,['Player','shooterID','season','Predicted.CS','TOI','P60','G60']]

player_season_stats_p1['season'] = (player_season_stats_p1['season'].astype(int) - 10001).astype(str)

player_season_stats_p1.columns = ['Player','shooterID','season','Predicted.CS_P1','TOI_P1','P60_P1','G60_P1']

player_season_stats = player_season_stats.merge(player_season_stats_p1, how='left', on=['Player','shooterID','season'])

player_season_stats = player_season_stats.dropna()

In [ ]:
import scipy

def rsquared(x, y):
    """ Return R^2 where x and y are array-like."""

    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    return r_value**2

In [ ]:
print rsquared(player_season_stats['P60'],player_season_stats['P60_P1'])
print rsquared(player_season_stats['G60'],player_season_stats['G60_P1'])
print rsquared(player_season_stats['Predicted.CS'],player_season_stats['Predicted.CS_P1'])
print rsquared(player_season_stats['TOI'],player_season_stats['TOI_P1'])

### Score Players

In [ ]:
def score_function(metric, season, num_comps, dissim_range, marcel_years, marcel_decay):


    try:
        player_comp_df_v3['comp_seasons_prior'] = player_comp_df_v3['PlayerSeason'].str[-4:].astype(int) - player_comp_df_v3['CompSeason'].str[-4:].astype(int)


        player_top_matches = player_comp_df_v3.loc[player_comp_df_v3['comp_seasons_prior'] >= 0, :].\
                    sort_values(['Player','PlayerID','PlayerSeason','PlayerScore','SimilarityScore'],ascending = True).\
                    groupby(['Player','PlayerID','PlayerSeason','PlayerScore']).head(num_comps)

        ## Keep last X years
        player_top_matches['player_seasons_prior'] = (int(season[-4:]) - player_top_matches['PlayerSeason'].str[-4:].astype(int))

        ## Keep Prior Season in Range
        player_top_matches_v2 = player_top_matches.loc[player_top_matches['player_seasons_prior'] < marcel_years,:].\
                                               loc[player_top_matches['player_seasons_prior'] >= 0,:]

        ## Create season weight
        player_top_matches_v2['season_weight'] = marcel_decay ** player_top_matches_v2['player_seasons_prior']

        ## Calculate weight
        player_top_matches_v2['comp_weight'] = ((dissim_range - player_top_matches_v2['SimilarityScore']) / dissim_range)\
                                      * player_top_matches_v2['season_weight']

        ## Weight metric
        player_top_matches_v2['weighted_metric'] = player_top_matches_v2['CompP1_' + metric]\
                                      * player_top_matches_v2['comp_weight']

        player_top_matches_v2['Season'] = season

        player_level = player_top_matches_v2.groupby(['Player','PlayerID','Season'], as_index=False)\
                                        ['weighted_metric','comp_weight'].sum()

        player_level['Projected_' + metric] = player_level['weighted_metric'] / player_level['comp_weight']
   
        return player_level.loc[:,['Player','PlayerID','Season','Projected_' + metric]]
    
    except Exception:
        pass
    

In [ ]:
scored_P60_2018 = score_function(metric = 'P60', season = '20162017', num_comps = 20, dissim_range = 40, marcel_years = 1, marcel_decay = 0.75)

In [ ]:
scored_P60_2018.sort_values(['Projected_P60'],ascending=False).head(20)

In [ ]:
print top_metrics

score_function(metric = 'P60_EV', 
               season = '20162017', 
               num_comps = 20, 
               dissim_range = 40, 
               marcel_years = 5, 
               marcel_decay = 0.33).sort_values(['Projected_P60_EV'],ascending=False).head(20)

In [ ]:
player_data_p18 = pd.DataFrame()

for i in range(top_metrics['rsquared'].count()):
    
    player_data = score_function(metric = top_metrics.iloc[i,1],
                           season = '20162017',
                           num_comps = top_metrics.iloc[i,2], 
                           dissim_range = top_metrics.iloc[i,3], 
                           marcel_years = top_metrics.iloc[i,4], 
                           marcel_decay = top_metrics.iloc[i,5])
    
    
    player_data_p18 = player_data_p18.append(player_data) ## store dataframes in list

In [ ]:
player_data_p182 = pd.melt(player_data_p18, id_vars=['Player','PlayerID','Season']).dropna()

player_projections_18 = pd.pivot_table(player_data_p182, index=['Player','PlayerID','Season'], columns=['variable'])
    
#levels = player_projections_18.columns.levels
#labels = player_projections_18.columns.labels
#player_projections_18.columns = levels[1][labels[1]]
#player_projections_18.reset_index(inplace=True)
print player_projections_18.columns

#print levels[1]

player_projections_18.columns = player_projections_18.columns.droplevel()
#del player_projections_18.index.name

print player_projections_18.columns

player_projections_18.head()

In [ ]:
player_projections_18_v2 = player_projections_18.join(current_roster,how='left', on = 'PlayerID')

player_projections_18_v2.head()

In [ ]:
player_projections_18['Projected_Score_Rank'] = player_projections_18['Projected_Score'].rank(ascending=0)
player_projections_18['G60_Rank'] = player_projections_18['Projected_G60'].rank(ascending=0)
player_projections_18['P60_Rank'] = player_projections_18['Projected_P60'].rank(ascending=0)

player_projections_18.head()

In [ ]:
player_projections_18.xs('CONNOR MCDAVID',level='Player')

In [53]:
df_vars = ['Player','PlayerSeason','Comparable','SimilarityScore','CompSeason','CompP1_Score','CompP1_P60','CompP1_G60']
player_comp_df_v3.loc[player_comp_df_v3['Player'] == 'MILAN LUCIC', df_vars]\
                .loc[player_comp_df_v3['PlayerSeason'] == '20162017', df_vars].head(20)

,Player,PlayerSeason,Comparable,SimilarityScore,CompSeason,CompP1_Score,CompP1_P60,CompP1_G60
61320,MILAN LUCIC,20162017,SIMON GAGNE,3.063667,20102011,67.808866,1.611162,0.663420
61321,MILAN LUCIC,20162017,MIKKEL BOEDKER,3.156084,20152016,50.741146,1.389616,0.545921
61322,MILAN LUCIC,20162017,PATRICK SHARP,3.297086,20152016,58.101475,1.456916,0.647518
61323,MILAN LUCIC,20162017,TOMAS FLEISCHMANN,3.439532,20112012,70.027814,2.368911,0.812198
61324,MILAN LUCIC,20162017,VINNY PROSPAL,3.626763,20112012,63.625908,2.240641,0.896256
61325,MILAN LUCIC,20162017,JAMES NEAL,3.629257,20152016,79.233550,1.811302,1.018857
61326,MILAN LUCIC,20162017,ANDREW BRUNETTE,3.730396,20082009,77.314471,2.630066,1.077896
61327,MILAN LUCIC,20162017,RAY WHITNEY,3.882514,20092010,76.404054,2.673350,0.802005
61328,MILAN LUCIC,20162017,ANDERS LEE,3.895657,20152016,73.086439,2.430317,1.589053
61329,MILAN LUCIC,20162017,KEITH TKACHUK,3.997534,20082009,59.217801,2.062225,0.837779
